# Wyjątki
## Generowanie wyjątków

Gdy wykonujemy operację, która może się nie udać (np. zapis/odczyt z dysku lub sieci) musimy być przygotowani, że rzuci ona wyjątek, czyli obiekt, który przekazuje informację o błędzie w górę, potencjalnie aż do zatrzymania wykonania programu. Gdy to my implementujemy taką operację, możemy sami rzucić wyjątek by dać znać, że zaszła sytuacja, która nie jest oczekiwana - inny sposób to byłoby zwracanie błędu jako jednego ze swoich wyników, jak w Go. Wtedy jednak muszą być one natychmiast obsłużone, mimo że sensowne miejsce gdzie chcielibyśmy tego dokonać może być w zupełnie innej warstwie aplikacji. Aby rzucić wyjątek należy:

In [ ]:
raise Exception()

lub:

In [ ]:
raise Exception

Drugi przypadek spowoduje stworzenie instancji implicite przy użyciu konstruktora bezparametrowego.
Zazwyczaj możemy również podać jakąś dodatkową informację:

In [ ]:
raise Exception("network is down")

## Blok try-except
Gdy wyjątek jest spodziewanym zachowaniem i nie chcemy z tego powodu zatrzymywać całego programu - np.: możemy się spodziewać utraty łączności sieciowej - możemy taki wyjątek "złapać":

In [ ]:
def ale_naprawde_otworz():
    open("nie_ma_takiego_pliku.txt", "r")
    
def otwieraj():
    ale_naprawde_otworz()

otwieraj()

# def ale_naprawde_otworz():
#     open("nie_ma_takiego_pliku.txt", "r")
#     print("po błędzie")
    
# def otwieraj():
#     ale_naprawde_otworz()
#     print("tu też nic")

    
# try:
#     otwieraj()
# except Exception as e:
#     if isinstance(FileNotFoundError, e):
#     print("Nic się nie stało, idź dalej")
# print("obsłużone")

# try:
#     f = open("nie_ma_takiego_pliku.txt", "r")
#     print(f.readlines())
# except FileNotFoundError as e:
#     print(e)
# finally:
#     f.close()

Blok kodu pod `finally` wykona się zawsze - niezależnie czy nastąpi wyjątek (nawet niespodziewany) czy nie. Blok po `else` tylko jeśli żaden z obsługiwanych w tym miejscu wyjątków nie nastapi - przydatne, zamiast dodawania dodatkowego kodu pod `try`, który mógłby przypadkowo rzucić jeden z obsługiwanych wyjątków z innego miejsca niż spodziewane. Po `except` następuje typ wyjątku - łapane i obsługiwane są wyjątki tego typu lub klas dziedziczących po nim. Można też łapać wszystkie wyjątki nie podając łapanego typu lub łąpiąc typ `BaseException` po którym wszystkie wyjątki dziedziczą, ale tracimy wtedy istotne informacje i jest to tzw. *code smell*. Blok kodu po `except` wykonuje się tylko gdy wyjątek określonego typu wystąpi. Takich klauzul może być wiele i jedna klauzula może obsługiwac więcej wyjatków:

In [ ]:
try:
    f = open("samp1le.txt", "r")
#     f = open("sample.txt", "r", encoding="no_such_encoding")
#     print(f.readlines())
    print(f.read())
#     f.write("whatev4r")
except FileNotFoundError | LookupError as e:
    print(e)
except UnsupportedOperation:
    print("File not open for writing - you can only read")
else:
    print(f"There are {len(f.readlines)} lines in this file")
finally:
    f.close()

Podczas obsługi wyjątku, np. na granicy warstwy w jakiejś warstwowej aplikacji, może się okazać, że poza zalogowaniem nie można danego wyjatku zignorować. Wtedy często opakowuje się bardziej "niskopoziomowy" wyjątek w bardziej "wysokopoziomowy" i rzuca dalej:

In [ ]:
import requests
import json

class StockApiException(Exception):
    pass

class ProductApiClient:
    
    def __init__(self, base_url):
        self.base_url = base_url
    
    def get_availability(self, product_id):
        try:
            result = requests.get(f"{self.base_url}/products/{product_id}/")
            return json.loads(result.content)
        except requests.exceptions.ConnectionError as e: 
            raise StockApiException(f"can't get product availability for {product_id}") from e
    
client = ProductApiClient("http://superfajnynieistniejącysklep.pl/")
client.get_availability(123)
            

## Standardowe wyjątki
Biblioteka standardowa obfituje w różne wbudowane typy wyjątków (opisane [tu](https://docs.python.org/3.9/library/exceptions.html#bltin-exceptions)). Wszystkie dziedziczą po `BaseException`, dzięki czemu potrafią przekazywać dalej *traceback* i akceptują argumenty, udostępniane potem przez pole `args`.
Hierarchia wbudowanych wyjątków wygląda tak:
```
BaseException
 +-- SystemExit
 +-- KeyboardInterrupt
 +-- GeneratorExit
 +-- Exception
      +-- StopIteration
      +-- StopAsyncIteration
      +-- ArithmeticError
      |    +-- FloatingPointError
      |    +-- OverflowError
      |    +-- ZeroDivisionError
      +-- AssertionError
      +-- AttributeError
      +-- BufferError
      +-- EOFError
      +-- ImportError
      |    +-- ModuleNotFoundError
      +-- LookupError
      |    +-- IndexError
      |    +-- KeyError
      +-- MemoryError
      +-- NameError
      |    +-- UnboundLocalError
      +-- OSError
      |    +-- BlockingIOError
      |    +-- ChildProcessError
      |    +-- ConnectionError
      |    |    +-- BrokenPipeError
      |    |    +-- ConnectionAbortedError
      |    |    +-- ConnectionRefusedError
      |    |    +-- ConnectionResetError
      |    +-- FileExistsError
      |    +-- FileNotFoundError
      |    +-- InterruptedError
      |    +-- IsADirectoryError
      |    +-- NotADirectoryError
      |    +-- PermissionError
      |    +-- ProcessLookupError
      |    +-- TimeoutError
      +-- ReferenceError
      +-- RuntimeError
      |    +-- NotImplemented
      tedError
      |    +-- RecursionError
      +-- SyntaxError
      |    +-- IndentationError
      |         +-- TabError
      +-- SystemError
      +-- TypeError
      +-- ValueError
      |    +-- UnicodeError
      |         +-- UnicodeDecodeError
      |         +-- UnicodeEncodeError
      |         +-- UnicodeTranslateError
      +-- Warning
           +-- DeprecationWarning
           +-- PendingDeprecationWarning
           +-- RuntimeWarning
           +-- SyntaxWarning
           +-- UserWarning
           +-- FutureWarning
           +-- ImportWarning
           +-- UnicodeWarning
           +-- BytesWarning
           +-- ResourceWarning
```

## Tworzenie własnych wyjątków
Aby utworzyć własny wyjątek należy odziedziczyć po typie `Exception`. Typ `BaseException` nie jest przeznaczony do dziedziczenia przez klasy użytkownika. Przykład:

In [ ]:
class ItemUnavailable(Exception):
    def __init__(self, item_id, reason, *args, **kwargs):
        self.__item_id = item_id
        self.__reason = reason
        super(ItemUnavailable, self).__init__(*args, **kwargs)
    
    def affected_item(self):
        return self.__item_id

    def why(self):
        return self.__reason

try:
    raise ItemUnavailable("1231221302938102", "Out of stock", "Can't get item")
except ItemUnavailable as e:
    print(e)
    print(e.affected_item())
    print(e.why())